# 1. Example alignment step (simplified):

In [ ]:
from osgeo import gdal

ref = r"path\to\DTM.rst"
src = r"path\to\PFac.tif"
tmp_aligned = r"path\to\PFac_aligned.rst"

ref_ds = gdal.Open(ref)
gt = ref_ds.GetGeoTransform()
proj = ref_ds.GetProjection()
W, H = ref_ds.RasterXSize, ref_ds.RasterYSize
xmin, xres, ymax, yres = gt[0], gt[1], gt[3], gt[5]
xmax = xmin + W * xres
ymin = ymax + H * yres

gdal.Warp(
    tmp_aligned, src,
    format="RST",
    dstSRS=proj,
    outputBounds=(xmin, ymin, xmax, ymax),
    xRes=abs(xres), yRes=abs(yres),
    width=W, height=H,
    resampleAlg="near",
    dstNodata=0,
    outputType=gdal.GDT_Float32,
    targetAlignedPixels=True
)


Done. Wrote: C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Output\BCR.rst


# 2. Clean and snap to valid P-factor classes
Once aligned, we cleaned the raster to remove invalid values and force all valid pixels to one of the three known P-factor classes.

In [ ]:
from osgeo import gdal
import numpy as np

in_rst  = r"path\to\PFac_aligned.rst"
out_rst = r"path\to\PFac_clean.rst"

# Open aligned raster
ds = gdal.Open(in_rst, gdal.GA_ReadOnly)
arr = ds.GetRasterBand(1).ReadAsArray().astype(np.float32)

# Step 1: Replace non-finite and out-of-range values with 0
arr[~np.isfinite(arr)] = 0
arr[(arr < 0) | (arr > 1)] = 0

# Step 2: Snap to known P classes
classes = np.array([0.02, 0.2, 0.6], dtype=np.float32)
mask = arr > 0  # exclude NoData
if mask.any():
    diffs = np.abs(arr[mask, None] - classes[None, :])
    arr[mask] = classes[np.argmin(diffs, axis=1)]

# Step 3: Write out as RST with Float32 and NoData=0
drv = gdal.GetDriverByName("RST")
out = drv.Create(out_rst, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)
out.SetGeoTransform(ds.GetGeoTransform())
out.SetProjection(ds.GetProjection())
b = out.GetRasterBand(1)
b.WriteArray(arr)
b.SetNoDataValue(0)
b.FlushCache()

out, ds = None, None
print(f"Wrote: {out_rst}")


# 3. Validation

In [ ]:
ds = gdal.Open(out_rst)
band = ds.GetRasterBand(1)
print("Min:", band.GetMinimum())
print("Max:", band.GetMaximum())
print("NoData:", band.GetNoDataValue())


# 4. Expected:

Min ≈ 0.02

Max ≈ 0.6

NoData = 0

Pixel dimensions, CRS, and extent match DTM.rst

# 5. Lessons Learned

Always align categorical rasters with nearest resampling.

Set both pixel values and metadata for NoData consistently.

Watch out for integer + scale/offset in GeoTIFFs — convert to Float32 before alignment.

Snap to discrete classes after all reprojection/resampling to prevent mid-values from creeping in.